In [1]:
# Imports
import pykep as pk
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep import AU, DAY2SEC
import pygmo as pg
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")

import numpy as np
import math

In [2]:
# Load planets
venus = pk.planet.spice('VENUS' , 'SUN', 'ECLIPJ2000', 'NONE')
earth = pk.planet.spice('EARTH', 'SUN', 'ECLIPJ2000', 'NONE' )

# Set launch window
start_date = pk.epoch_from_string("2035-01-01 00:00:00")
end_date = pk.epoch_from_string("2040-12-31 00:00:00")

# Parameters
launch_step = 30     # days between launch trials
tof_range = (100, 300)  # in days, time of flight range

In [4]:
# Results list
results = []

# Scan through departure dates
t_dep = start_date
while t_dep.mjd2000 < end_date.mjd2000:
    rE, vE = earth.eph(t_dep)

    for tof_days in range(tof_range[0], tof_range[1], 30):  # try multiple TOFs
        try:
            t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
            rV, vV = venus.eph(t_arr)

            dt_sec = tof_days * pk.DAY2SEC
            l = pk.lambert_problem(r1=rE, r2=rV, tof=dt_sec, mu=pk.MU_SUN, max_revs=0)

            v1 = np.array(l.get_v1()[0])
            v2 = np.array(l.get_v2()[0])
            v_inf_earth = (v1 - np.array(vE))/1000 # km/s; vector
            delta_v = (np.linalg.norm(v1 - np.array(vE)))/1000 #km/s 
            v_arrival   = l.get_v2()[0] # velocity of spacecraft at jupiter arrival
            v_inf_venus_vec   = (np.array(v_arrival) - np.array(vV))/1000   # km/s
            v_inf_venus = np.linalg.norm(v_inf_venus_vec)

            # Calculating the Incoming Earth - Venus Asymptote
            
            C3 = np.linalg.norm(v_inf_earth) ** 2  # in km²/s², C3 of the incoming asymptote/ C3 required during launch from Earth
            RLA = np.degrees(math.atan((v_inf_earth[1])/(v_inf_earth[0]))) # Right ascension of incoming asymptote
            DLA = np.degrees(math.atan((v_inf_earth[2])/(np.linalg.norm(v_inf_earth)))) # Declination of incoming asymptote

            # Calculating Flyby speed

            rp = 10 * 6051.8e3 # m (10rVenus) # radius of periapsis for the flyby trajectory (change this)

            mu = 6.67e-11 * 0.32486e15 # m3/s2  # gravitational parameter for venus

            v_in = v_inf_venus * 1000 # m/s # Venus arrival speed

            # Turning Angle
            phi = np.degrees(np.pi - 2 * np.acos (1/(((v_in**2)*rp)/mu + 1)))
            # print (np.pi) # Degree/radian check
            # print (f" Turning angle: {phi:.2f} degrees") # degrees


            # Law of Cosines to find the v_out
            #print(vJ) # vector/scalar check
            vV_mag = np.linalg.norm(vV)
            #print(vJ_mag)

            exit_speed = (math.sqrt((vV_mag**2) + (v_in**2) - 2*vV_mag*v_in*np.cos(phi)))/1000 # km/s
            #print(f"Flyby speed: {(exit_speed - hyperbolic_v_arrival):.2f} km/s")
            #print (f"Jupiter Departure Speed: {exit_speed:.2f} km/s")

            # Time to Heliopause from Jupiter flyby

            #d = 201.44*AU # 200 AU in meters
            #s = exit_speed*1000 # speed of space craft at jupiter exit in m/s 

            # print (d) .pk check

            #time_to_200AU = (d/s)*SEC2DAY*DAY2YEAR
            #total_time = time_to_200AU + tof_days/365
            flyby_speed = exit_speed - v_inf_venus

            #print(f"Time to reach Jupiter: {dt*SEC2DAY*DAY2YEAR: .2f} years")
            #print(f"Time to 200 AU from Jupiter Flyby: {time_to_200AU:.2f} years")
            #print(f"Total Time to reach 200 AU from launch: {time_to_200AU + dt*SEC2DAY*DAY2YEAR : .2f} years")

            #d1 = 91.44*AU
            #s1 = exit_speed*1000

            #time_to_90AU = (d1/s1)*SEC2DAY*DAY2YEAR # time to reach 90 AU

            #d2 = 121.44*AU
            #s2 = exit_speed*1000

            #time_to_120AU = (d2/s2)*SEC2DAY*DAY2YEAR # time to reach 120 AU
            

            
            
            if C3 < 80:

                # Get Keplerian elements of transfer orbit
                kep = pk.ic2par(rE, v1, pk.MU_SUN)  # r, v at arrival → orbital elements

                results.append({
                'departure': str(t_dep),
                'arrival': str(t_arr),
                'tof_days': tof_days,
                'C3': C3,
                'delta_v': delta_v,
                'sma': kep[0],   # semi-major axis
                'ecc': kep[1],   # eccentricity
                'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                'raan': np.degrees(kep[3]),
                'argp': np.degrees(kep[4]),
                'ta': np.degrees(kep[5]),
                'turning angle': phi,
                'flyby speed': flyby_speed,
                'venus departure speed': exit_speed,
                'v_inf_earth': v_inf_earth,
                'RLA': RLA,
                'DLA': DLA,
                    })

        except RuntimeError as e:
            # Lambert solver may fail for some configurations; skip them
            continue

    t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

In [10]:
# Sort by C3 (ascending)
results.sort(key=lambda x: x['C3'])

# Print trajectories
for res in results[:]:
    print(f"Launch: {res['departure']}, Arrival: {res['arrival']}, C3: {res['C3']:.2f} km²/s², Delta v: {res['delta_v']:.2f} km/s \n")
    print(f"Time to reach Venus: {res['tof_days']/365:.2f} years\n")
    print(f"Elements for Earth -> Venus → SMA: {res['sma']/1000:.3f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°\n")
    print(f"Elements for Earth -> Venus → RA: {res['raan']:.2f}°, AP: {res['argp']:.2f}°, TA: {res['ta']:.2f}°\n\n\n")

Launch: 2039-May-10 00:00:00, Arrival: 2039-Oct-17 00:00:00, C3: 7.06 km²/s², Delta v: 2.66 km/s 

Time to reach Venus: 0.44 years

Elements for Earth -> Venus → SMA: 129389268.103 km, ECC: 0.1684, INC: 2.12°

Elements for Earth -> Venus → RA: 228.57°, AP: 185.94°, TA: 173.10°



Launch: 2040-Dec-30 00:00:00, Arrival: 2041-May-09 00:00:00, C3: 7.39 km²/s², Delta v: 2.72 km/s 

Time to reach Venus: 0.36 years

Elements for Earth -> Venus → SMA: 127904187.548 km, ECC: 0.1521, INC: 0.74°

Elements for Earth -> Venus → RA: 277.93°, AP: 352.51°, TA: -170.81°



Launch: 2037-Oct-17 00:00:00, Arrival: 2038-Mar-26 00:00:00, C3: 8.92 km²/s², Delta v: 2.99 km/s 

Time to reach Venus: 0.44 years

Elements for Earth -> Venus → SMA: 127497069.330 km, ECC: 0.1696, INC: 2.54°

Elements for Earth -> Venus → RA: 203.40°, AP: 358.00°, TA: -177.57°



Launch: 2040-Nov-30 00:00:00, Arrival: 2041-May-09 00:00:00, C3: 9.12 km²/s², Delta v: 3.02 km/s 

Time to reach Venus: 0.44 years

Elements for Earth -> V